In [ ]:
import hydra
import hydra.experimental
import numpy as np
import tqdm
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import RobertaTokenizer
from omegaconf import DictConfig

from torchfly.text.decode import TransformerDecoder
from torchfly.common import set_random_seed, move_to_device

from configure_dataloader import DataLoaderHandler
from model import Generator, TextGAILModel

import logging

In [ ]:
random_seed = 1
set_random_seed(random_seed)

In [ ]:
hydra.experimental.initialize("config")

In [ ]:
config = hydra.experimental.compose("config.yaml")
print(config.pretty())

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
dataloader_handler = DataLoaderHandler(config)
valid_dataloader = dataloader_handler.test_dataloader(config)
collate_fn = test_dataloader.dataset.collate_fn

In [ ]:
device = torch.device("cuda")

In [ ]:
model = TextGAILModel(config)
model = model.cuda()

In [ ]:
decoder = TransformerDecoder(config.decode)
decoder.register_generator(model.generator.decoder)
decoder.register_tokenizer(tokenizer)
decoder.prepare_model_inputs_for_generation = model.generator.prepare_model_inputs_for_generation

## MLE Generation

In [ ]:
# location to store results
os.makedirs(config.task.name,exist_ok=True)

In [ ]:
print(config.task.mle_weights_path)

In [ ]:
mle_weights = torch.load(config.task.mle_weights_path)
model.generator.load_state_dict(mle_weights)

In [ ]:
temperatures = (np.arange(10) + 1) / 10.0

In [ ]:
for temperature in temperatures:
    f_write = open(f"{config.task.name}/mle_{temperature}_{random_seed}.txt", "w")

    for batch in tqdm.tqdm(valid_dataloader):
        batch = collate_fn(batch)
        batch = move_to_device(batch, device)

        ground_truth = batch["target_text"]

        results = decoder.generate(batch["source_token_ids"], temperature=temperature)
        generated = []

        for i in range(len(results["tokens"])):
            res = tokenizer.decode(results["tokens"][i][0][1:-1].tolist())
            generated.append(res)

        for gt, gen in zip(ground_truth, generated):
            f_write.write(json.dumps([gt, gen]))
            f_write.write("\n")
        
    f_write.close()

### Beam Search

In [ ]:
# f_write = open(f"{config.task.name}/mle_beam_4.txt", "w")

# for batch in tqdm.tqdm(valid_dataloader):
#     batch = collate_fn(batch)
#     batch = move_to_device(batch, device)

#     ground_truth = batch["target_text"]

#     results = decoder.generate(batch["source_token_ids"], do_sample=False, num_beams=4)
#     generated = []

#     for i in range(len(results["tokens"])):
#         res = tokenizer.decode(results["tokens"][i][0][1:-1].tolist())
#         generated.append(res)

#     for gt, gen in zip(ground_truth, generated):
#         f_write.write(json.dumps([gt, gen]))
#         f_write.write("\n")        

# f_write.close()

## TextGAIL Generation

In [ ]:
print(config.task.textgail_weights_path)

In [ ]:
textgail_weights = torch.load(config.task.textgail_weights_path)
model.load_state_dict(textgail_weights)

In [ ]:
for temperature in temperatures:
    f_write = open(f"{config.task.name}/textgail_{temperature}_{random_seed}.txt", "w")

    for batch in tqdm.tqdm(valid_dataloader):
        batch = collate_fn(batch)
        batch = move_to_device(batch, device)

        ground_truth = batch["target_text"]

        results = decoder.generate(batch["source_token_ids"], temperature=temperature)
        generated = []

        for i in range(len(results["tokens"])):
            res = tokenizer.decode(results["tokens"][i][0][1:-1].tolist())
            generated.append(res)

        for gt, gen in zip(ground_truth, generated):
            f_write.write(json.dumps([gt, gen]))
            f_write.write("\n")
        
    f_write.close()

### Beam Search

In [ ]:
f_write = open(f"{config.task.name}/textgail_no_pretrain2_beam_4.txt", "w")

for batch in tqdm.tqdm(valid_dataloader):
    batch = collate_fn(batch)
    batch = move_to_device(batch, device)

    ground_truth = batch["target_text"]

    results = decoder.generate(batch["source_token_ids"], do_sample=False, num_beams=4)
    generated = []

    for i in range(len(results["tokens"])):
        res = tokenizer.decode(results["tokens"][i][0][1:-1].tolist())
        generated.append(res)
        
    for gt, gen in zip(ground_truth, generated):
        f_write.write(json.dumps([gt, gen]))
        f_write.write("\n")        

f_write.close()